In [8]:
# PyPDF2
from PyPDF2 import PdfReader
import pandas as pd

In [9]:
reader = PdfReader("./resume_pdf/cv_sak.pdf")
number_of_pages = len(reader.pages)
page = reader.pages[0]
text = page.extract_text()

In [10]:
number_of_pages

1

In [11]:
page

{'/Type': '/Page',
 '/Resources': {'/ProcSet': ['/PDF', '/Text', '/ImageB', '/ImageC', '/ImageI'],
  '/ExtGState': {'/G3': {'/ca': 1, '/BM': '/Normal'},
   '/G7': {'/CA': 1,
    '/ca': 1,
    '/LC': 0,
    '/LJ': 0,
    '/LW': 4,
    '/ML': 4,
    '/SA': True,
    '/BM': '/Normal'}},
  '/Font': {'/F4': {'/Type': '/Font',
    '/Subtype': '/Type0',
    '/BaseFont': '/AAAAAA+Gotham-Bold',
    '/Encoding': '/Identity-H',
    '/DescendantFonts': [IndirectObject(226, 0, 5904228800)],
    '/ToUnicode': {'/Filter': '/FlateDecode'}},
   '/F5': {'/Type': '/Font',
    '/Subtype': '/Type0',
    '/BaseFont': '/BAAAAA+CanvaSans-Bold',
    '/Encoding': '/Identity-H',
    '/DescendantFonts': [IndirectObject(228, 0, 5904228800)],
    '/ToUnicode': {'/Filter': '/FlateDecode'}},
   '/F6': {'/Type': '/Font',
    '/Subtype': '/Type0',
    '/BaseFont': '/CAAAAA+CanvaSans-Regular',
    '/Encoding': '/Identity-H',
    '/DescendantFonts': [IndirectObject(230, 0, 5904228800)],
    '/ToUnicode': {'/Filter': '/Fl

In [12]:
text

'Thear Sithysak\nComputer Science(Data Science)\nSt 618, Sangkat Boeng Kak Ti 2,\nKhan Toul Kark, Phnom Penh \n(855)10-382-829\nsithysak.thear@student.cadt.edu.khCONTACT\nEXPERIENCE\nCapstone project\nMarch – June 2023\nManaged to design Rental Car website as\nteam \n. Design and do frontend\n. Write report \nUI/UX Design\n 1 – 24 March 2023\nFinal project for web application course. \nI designed about School Management as a\nfinal project.\nEntrepreneurship\n 1 – 15 July 2023\nResearch on E-Marketing by research data\non internet and make surveys from seller\nand customer.EDUCATION\nBachelor of Computer Science at\nCambodia Academy of Digital\nTechnology (CADT)  \nPaññāsāstra University of\nCambodia(PUC) part time English\nclass   (2017-2019)\nHigh school (2019-2021)-Grand D\nSKILL\nEnvironment conversation\nReading\nSport\nMusicINTERESTS. C\n. Python\n. Database, SQL\n. PHP (Laravel)\n. GitPERSONAL INFORMATION\nGender : Male \nNationality : Cambodian \nDate of Birth : 17th February 1

In [13]:
split_text = text.split('\n')
split_text

['Thear Sithysak',
 'Computer Science(Data Science)',
 'St 618, Sangkat Boeng Kak Ti 2,',
 'Khan Toul Kark, Phnom Penh ',
 '(855)10-382-829',
 'sithysak.thear@student.cadt.edu.khCONTACT',
 'EXPERIENCE',
 'Capstone project',
 'March – June 2023',
 'Managed to design Rental Car website as',
 'team ',
 '. Design and do frontend',
 '. Write report ',
 'UI/UX Design',
 ' 1 – 24 March 2023',
 'Final project for web application course. ',
 'I designed about School Management as a',
 'final project.',
 'Entrepreneurship',
 ' 1 – 15 July 2023',
 'Research on E-Marketing by research data',
 'on internet and make surveys from seller',
 'and customer.EDUCATION',
 'Bachelor of Computer Science at',
 'Cambodia Academy of Digital',
 'Technology (CADT)  ',
 'Paññāsāstra University of',
 'Cambodia(PUC) part time English',
 'class   (2017-2019)',
 'High school (2019-2021)-Grand D',
 'SKILL',
 'Environment conversation',
 'Reading',
 'Sport',
 'MusicINTERESTS. C',
 '. Python',
 '. Database, SQL',
 '. PHP